# Download Data for Dash app

Needs png with transparent background and the coordinates, to be able to plug it in in the map.

In [1]:
%matplotlib widget
import matplotlib.pylab as plt
import seaborn as sns

import numpy as np

import pandas as pd

import re

from tqdm import tqdm
import glob
import os

from owslib.wms import WebMapService
import rasterio

## Floading maps 2020

For Denmark: https://gis.nst.dk/server/rest/services/ekstern

The newest ones (Plantrin2) are divided in 14 zones. Although they have more scenarios (events happening in 1/20, 1/100 years and extreme) they do not include most of Denmark.

In [2]:
df_flood_maps = pd.read_csv('Plantrin_2.csv')
df_flood_maps.head(20)

,zone,url
0,Aabenraa,https://gis.nst.dk/server/services/ekstern/Aab...
1,Esbjerg,https://gis.nst.dk/server/services/ekstern/Esb...
2,Fredericia,https://gis.nst.dk/server/services/ekstern/Fre...
3,Holstebro,https://gis.nst.dk/server/services/ekstern/Hol...
4,Juelsminde,https://gis.nst.dk/server/services/ekstern/Jue...
5,Kobenhavn_NS,https://gis.nst.dk/server/services/ekstern/Kob...
6,Kobenhavn_Nord,https://gis.nst.dk/server/services/ekstern/Koe...
7,Kobenhavn_Syd,https://gis.nst.dk/server/services/ekstern/Koe...
8,Kolding,https://gis.nst.dk/server/services/ekstern/Kol...
9,Korsoer,https://gis.nst.dk/server/services/ekstern/Kor...


In [3]:
# Quick fix. These two are not in WMS format. Needs to be fixed later
df_flood_maps.drop([6,7], inplace=True);

## Check what layers are available

In [4]:
wms = WebMapService(df_flood_maps.iloc[3].url+'/WmsServer')

for layer_nb in wms.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

0                             Risikoområde ['EPSG:25832', 'EPSG:4326']
3 Kulturarv_Seværdige_Fortidsminder_punkter ['EPSG:25832', 'EPSG:4326']
4  Kulturarv_Seværdige_Fortidsminder_areal ['EPSG:25832', 'EPSG:4326']
5                         Kulturarv_Kirker ['EPSG:25832', 'EPSG:4326']
6              Kulturarv_Fredede_Bygninger ['EPSG:25832', 'EPSG:4326']
7          Kulturarv_Fortidsminder_punkter ['EPSG:25832', 'EPSG:4326']
8            Kulturarv_Fortidsminder_areal ['EPSG:25832', 'EPSG:4326']
9                       Bes_sten_jorddiger ['EPSG:25832', 'EPSG:4326']
10                       Begravelsesomraade ['EPSG:25832', 'EPSG:4326']
12                         Vandvaerksboring ['EPSG:25832', 'EPSG:4326']
13    Miljoeinteresser_Vandforsyningsboring ['EPSG:25832', 'EPSG:4326']
14              Miljoeinteresser_Natura2000 ['EPSG:25832', 'EPSG:4326']
15         Miljoeinteresser_Beskyttet_natur ['EPSG:25832', 'EPSG:4326']
16    Miljoeinteresser_Aktiv_vandindvinding ['EPSG:25832', 'EPSG:4326']

In [5]:
wms = WebMapService(df_flood_maps.iloc[0].url+'/WmsServer')

for layer_nb in wms.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

0                             Risikoområde ['EPSG:25832', 'EPSG:4326']
3 Kulturarv_Seværdige_Fortidsminder_punkter ['EPSG:25832', 'EPSG:4326']
4  Kulturarv_Seværdige_Fortidsminder_areal ['EPSG:25832', 'EPSG:4326']
5                         Kulturarv_Kirker ['EPSG:25832', 'EPSG:4326']
6              Kulturarv_Fredede_Bygninger ['EPSG:25832', 'EPSG:4326']
7          Kulturarv_Fortidsminder_punkter ['EPSG:25832', 'EPSG:4326']
8            Kulturarv_Fortidsminder_areal ['EPSG:25832', 'EPSG:4326']
9                       Bes_sten_jorddiger ['EPSG:25832', 'EPSG:4326']
10                       Begravelsesomraade ['EPSG:25832', 'EPSG:4326']
12                         Vandvaerksboring ['EPSG:25832', 'EPSG:4326']
13    Miljoeinteresser_Vandforsyningsboring ['EPSG:25832', 'EPSG:4326']
14         Miljoeinteresser_Beskyttet_natur ['EPSG:25832', 'EPSG:4326']
15    Miljoeinteresser_Aktiv_vandindvinding ['EPSG:25832', 'EPSG:4326']
17        Aabenraa_forurenende_virksomheder ['EPSG:25832', 'EPSG:4326']

We'll be interested in the *Udbredelse* layers for the diferent scenarios:

    Udbredelse_AAB_MT20_2019
    Udbredelse_AAB_MT100_2115
    Udbredelse_AAB_MT100_2065
    Udbredelse_AAB_MTEXT_2019
    Udbredelse_AAB_MTEXT_2115
    Udbredelse_AAB_MT100_2019
    
Since they can be in different layer numbers for the different zones, we'll check for each one

In [6]:
def check_layer_number(url,layer_name):
    wms = WebMapService(url+'/WmsServer')
    layer_titles = [wms[layer_nb].title for layer_nb in wms.contents]
    layer_number = [layer_nb for layer_nb in wms.contents if re.match(layer_name,wms[layer_nb].title) ]
    return layer_number

In [7]:
scenarios = ['MTEXT_2115',
             'MTEXT_2019',
             'MT100_2115',
             'MT100_2065',
             'MT100_2019',
             'MT20_2019',
             'MT50_2115',
             'MT50_2065',
             'MT50_2019',
            ]

for scenario in scenarios:
    print(scenario)
    df_flood_maps['Udbredelse_'+scenario] = df_flood_maps.url.apply(lambda x: check_layer_number(x,'Udbredelse_\w+_'+scenario))

MTEXT_2115
MTEXT_2019
MT100_2115
MT100_2065
MT100_2019
MT20_2019
MT50_2115
MT50_2065
MT50_2019


In [8]:
df_flood_maps.head(100)

,zone,url,Udbredelse_MTEXT_2115,Udbredelse_MTEXT_2019,Udbredelse_MT100_2115,Udbredelse_MT100_2065,Udbredelse_MT100_2019,Udbredelse_MT20_2019,Udbredelse_MT50_2115,Udbredelse_MT50_2065,Udbredelse_MT50_2019
0,Aabenraa,https://gis.nst.dk/server/services/ekstern/Aab...,[95],[98],[96],[97],[99],[100],[],[],[]
1,Esbjerg,https://gis.nst.dk/server/services/ekstern/Esb...,[97],[100],[98],[99],[101],[102],[],[],[]
2,Fredericia,https://gis.nst.dk/server/services/ekstern/Fre...,[92],[95],[93],[94],[96],[97],[],[],[]
3,Holstebro,https://gis.nst.dk/server/services/ekstern/Hol...,[],[],[96],[],[99],[101],[97],[98],[100]
4,Juelsminde,https://gis.nst.dk/server/services/ekstern/Jue...,[95],[98],[96],[97],[99],[100],[],[],[]
5,Kobenhavn_NS,https://gis.nst.dk/server/services/ekstern/Kob...,"[95, 209]","[98, 212]","[96, 210]","[97, 211]","[99, 213]","[100, 214]",[],[],[]
8,Kolding,https://gis.nst.dk/server/services/ekstern/Kol...,[104],[108],[105],[106],[109],[110],[],[],[]
9,Korsoer,https://gis.nst.dk/server/services/ekstern/Kor...,[96],[99],[97],[98],[100],[101],[],[],[]
10,Koege,https://gis.nst.dk/server/services/ekstern/Koe...,[95],[98],[96],[97],[99],[100],[],[],[]
11,Nyborg,https://gis.nst.dk/server/services/ekstern/Nyb...,[97],[100],[98],[99],[101],[102],[],[],[]


## Download data and save it as transparent png and get the coordinates

In [9]:
def download_image(url,layer,size,crs,dir_path):
    
    wms = WebMapService(url+'/WmsServer')
    srs = wms[layer].crsOptions
    bbox = wms[layer].boundingBoxWGS84 
    output_name = wms[layer].title
        
    if crs not in srs:
        print('CRS not available (available options:)',srs)
        pass
        
    # Get the image from the server
    img = wms.getmap(layers=[layer],
                     styles=['default'],
                     srs=crs,
                     bbox=bbox,
                     size=size,
                     transparent=True,
                     format='image/png8'
                    )
    
    # Save the array as an image
    print('Saved', dir_path+output_name+'.png')
    with open(dir_path+output_name+'.png', 'wb') as out:
        out.write(img.read())
        
    return dir_path+output_name+'.png',bbox

In [10]:
scenarios = ['MTEXT_2115',
             'MTEXT_2019',
             'MT100_2115',
             'MT100_2065',
             'MT100_2019',
             'MT20_2019',
             'MT50_2115',
             'MT50_2065',
             'MT50_2019',
            ]

In [11]:
file_scenario_list = []
file_name_list = []
bbox_list = []

for scenario in tqdm(scenarios):
    
    # Create path
    dir_path = 'ForDash/Udbredelse_'+scenario+'/'
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
    # Create empty column for the bbox and image name to be updated during download
    df_flood_maps['bbox_'+scenario] = 0
    df_flood_maps['png_file_'+scenario] = 0
    
    for idx,row in df_flood_maps.iterrows():
        if len(row['Udbredelse_'+scenario]) == 0:
            print('Scenario for ',row['Udbredelse_'+scenario],'does not exist')
            continue
        else:
            # there can be more than one layer (eg. cph north and south)
            for layer in row['Udbredelse_'+scenario]:
                file_name,bbox = download_image(row['url'],
                                                  layer=layer,
                                                  size=(4000,4000),
                                                  crs='EPSG:4326',
                                                  dir_path=dir_path)
                file_name_list.append(file_name)
                bbox_list.append(bbox)
                file_scenario_list.append(scenario)
                

  0%|          | 0/9 [00:00<?, ?it/s]

Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_AAB_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_ESB_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_FRE_MTEXT_2115.png
Scenario for  [] does not exist
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_JUE_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_KHS_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_KHN_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_KOL_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_KOR_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_KBU_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_NYB_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_ODE_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_RAF_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_SKB_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_SYD_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2115/Udbredel

 11%|█         | 1/9 [01:11<09:33, 71.73s/it]

Saved ForDash/Udbredelse_MTEXT_2115/Udbredelse_VOR_MTEXT_2115.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_AAB_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_ESB_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_FRE_MTEXT_2019.png
Scenario for  [] does not exist
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_JUE_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_KHS_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_KHN_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_KOL_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_KOR_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_KBU_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_NYB_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_ODE_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_RAF_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredelse_SKB_MTEXT_2019.png
Saved ForDash/Udbredelse_MTEXT_2019/Udbredel

 22%|██▏       | 2/9 [02:14<08:02, 68.97s/it]

...

Saved ForDash/Udbredelse_MT50_2115/Udbredelse_HOL_MT50_2115.png
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist


 89%|████████▉ | 8/9 [05:50<00:28, 28.01s/it]

Saved ForDash/Udbredelse_MT50_2065/Udbredelse_HOL_MT50_2065.png
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist


100%|██████████| 9/9 [05:52<00:00, 39.12s/it]

Saved ForDash/Udbredelse_MT50_2019/Udbredelse_HOL_MT50_2019.png
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist


## Data from 2019

This is for only one scenario (1/1000 years floods) but includes all Denmark


https://gis.nst.dk/server/services/ekstern/OSD_2016_2021_Plantrin1_Endelige_data/MapServer/WMSServer?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapabilities

 
There are several layers in this one, but the two that covers all of Denmark for an extreme flooding is

 

-          Fare fra hav (ekstrem hændelse)

-          Fare fra vandløb (1000 år)

In [12]:
url_2019 = 'https://gis.nst.dk/server/services/ekstern/OSD_2016_2021_Plantrin1_Endelige_data/MapServer'
wms_2019= WebMapService(url_2019+'/WMSServer')

for layer_nb in wms_2019.contents:
    print('%s %40s %s'%(layer_nb,wms_2019[layer_nb].title,wms_2019[layer_nb].crsOptions))

0                         Hovedvandoplande ['EPSG:25832', 'EPSG:4326']
1                     Vandområdedistrekter ['EPSG:25832', 'EPSG:4326']
2                           Kommunegrænser ['EPSG:25832', 'EPSG:4326']
3      Potentielt Forurenende Virksomheder ['EPSG:25832', 'EPSG:4326']
4                      Økonomisk Aktivitet ['EPSG:25832', 'EPSG:4326']
5                                Kulturarv ['EPSG:25832', 'EPSG:4326']
6                    Kritisk Infrastruktur ['EPSG:25832', 'EPSG:4326']
7                            Infrastruktur ['EPSG:25832', 'EPSG:4326']
8                                Beredskab ['EPSG:25832', 'EPSG:4326']
9                               Befolkning ['EPSG:25832', 'EPSG:4326']
10                          Arealanvendelse ['EPSG:25832', 'EPSG:4326']
11               Fare fra vandløb (1000 år) ['EPSG:25832', 'EPSG:4326']
12          Fare fra hav (ekstrem hændelse) ['EPSG:25832', 'EPSG:4326']
13                         Samlet sårbarhed ['EPSG:25832', 'EPSG:4326']
14

In [17]:
# Create path
dir_path = 'ForDash/FloodingDK2019/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

file_name,bbox = download_image(url_2019,layer='11',size=(4000,4000),crs='EPSG:4326',dir_path='ForDash/FloodingDK2019/')
file_scenario_list.append('2019')
file_name_list.append(file_name)
bbox_list.append(bbox)
                                
file_name,bbox = download_image(url_2019,layer='12',size=(4000,4000),crs='EPSG:4326',dir_path='ForDash/FloodingDK2019/')
file_scenario_list.append('2019')
file_name_list.append(file_name)
bbox_list.append(bbox)                               

Saved ForDash/FloodingDK2019/Fare fra vandløb (1000 år).png
Saved ForDash/FloodingDK2019/Fare fra hav (ekstrem hændelse).png


## Put all info in a dataframe

In [20]:
df_png = pd.DataFrame({'scenario':file_scenario_list,
                       'file':file_name,
                       'bbox':bbox_list})
df_png.to_csv('png_file_list')
df_png.tail(100)

,scenario,file,bbox
6,MTEXT_2115,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(9.377683, 55.47553, 9.553077, 55.510132)"
7,MTEXT_2115,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(11.090517, 55.307539, 11.217802, 55.363792)"
8,MTEXT_2115,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(12.120189, 55.410787, 12.468536, 55.639661)"
9,MTEXT_2115,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(10.780247, 55.284468, 10.857497, 55.321463)"
10,MTEXT_2115,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(10.356789, 55.390362, 10.653339, 55.57148)"
...,...,...,...
101,MT50_2115,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(8.590139, 56.34769, 8.638287, 56.360013)"
102,MT50_2065,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(8.590139, 56.34769, 8.638287, 56.360013)"
103,MT50_2019,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(8.59072, 56.348062, 8.638312, 56.36004)"
104,2019,ForDash/FloodingDK2019/Fare fra hav (ekstrem h...,"(8.019848, 54.445098, 15.562142, 57.653192)"
